In [5]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

In [6]:
!pip install tensorflow==1.14
!pip install fastBPE
!pip install transformers
!pip install gsutil

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [25]:
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)

In [3]:
!wget -O moby_dick.txt https://www.gutenberg.org/files/2701/2701-0.txt

--2020-12-18 14:06:42--  https://www.gutenberg.org/files/2701/2701-0.txt
Resolving www.gutenberg.org... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276201 (1.2M) [text/plain]
Saving to: ‘moby_dick.txt’

moby_dick.txt       100%[===================>]   1.22M  1.71MB/s    in 0.7s    

2020-12-18 14:06:43 (1.71 MB/s) - ‘moby_dick.txt’ saved [1276201/1276201]



In [3]:
!wget -O vocab https://raw.githubusercontent.com/salesforce/ctrl/master/vocab
!mv vocab ../
!wget -O codes https://raw.githubusercontent.com/salesforce/ctrl/master/codes
!mv codes ../
!wget -O generation.py https://raw.githubusercontent.com/salesforce/ctrl/master/generation.py
!mv generation.py ../
!wget -O convert_tf_to_huggingface_pytorch.py https://raw.githubusercontent.com/salesforce/ctrl/master/convert_tf_to_huggingface_pytorch.py
!wget -O make_tf_records.py https://raw.githubusercontent.com/salesforce/ctrl/master/training_utils/make_tf_records.py

--2020-12-19 09:22:30--  https://raw.githubusercontent.com/salesforce/ctrl/master/vocab
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2996416 (2.9M) [text/plain]
Saving to: ‘vocab’

vocab               100%[===================>]   2.86M  --.-KB/s    in 0.06s   

2020-12-19 09:22:30 (45.6 MB/s) - ‘vocab’ saved [2996416/2996416]

--2020-12-19 09:22:32--  https://raw.githubusercontent.com/salesforce/ctrl/master/codes
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3332497 (3.2M) [text/plain]
Saving to: ‘codes’

codes               100%[===================>]   3.18M  --.-KB/s    in 0.08s   

2020-12-19 09:22:32 (38.8 MB/s) - ‘codes’ saved [3332497/3332497]

--2020-12-19 09:22:33--  https://raw.githubusercontent.com/salesfo

In [4]:
!python make_tf_records.py --text_file moby_dick.txt --control_code Moby --sequence_len 256

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

In [4]:
!gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt .

Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.meta...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/checkpoint...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.index...
| [4/4 files][ 12.7 GiB/ 12.7 GiB] 100% Done  22.4 MiB/s ETA 00:00:00           
Operation completed over 4 objects/12.7 GiB.                                     


In [70]:
!cp /anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/keras.py ./original

In [73]:
!ls ./original

./original


In [47]:
!echo $PATH

/anaconda/envs/py37_pytorch/bin:/anaconda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [74]:
!wget -O training.py https://raw.githubusercontent.com/salesforce/ctrl/master/training_utils/training.py
!wget -O transformer.py https://raw.githubusercontent.com/salesforce/ctrl/master/transformer.py
#!wget -O keras.py https://raw.githubusercontent.com/tensorflow/estimator/master/tensorflow_estimator/python/estimator/keras.py
!wget -O estimator.patch https://raw.githubusercontent.com/salesforce/ctrl/master/estimator.patch
#https://github.com/tensorflow/estimator/blob/master/tensorflow_estimator/python/estimator/keras.py
!patch -b /anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch
!python training.py --model_dir seqlen256_v1.ckpt/ --iterations 10

--2020-12-19 10:35:18--  https://raw.githubusercontent.com/salesforce/ctrl/master/training_utils/training.py
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6255 (6.1K) [text/plain]
Saving to: ‘training.py’

training.py         100%[===================>]   6.11K  --.-KB/s    in 0s      

2020-12-19 10:35:18 (80.4 MB/s) - ‘training.py’ saved [6255/6255]

--2020-12-19 10:35:18--  https://raw.githubusercontent.com/salesforce/ctrl/master/transformer.py
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4538 (4.4K) [text/plain]
Saving to: ‘transformer.py’

transformer.py      100%[===================>]   4.43K  --.-KB/s    in 0s      

2020-12-19 10:35:19 (60.1 MB/s) - ‘transformer.py’ saved [4538/4538]

--2020-12-19 10:35:19-- 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tied_embedding_softmax (TiedEmb multiple             315810054   input_1[0][0]                    
                                                                 encoder[0][0]                    
__________________________

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
2020-12-19 10:36:33.051514: I tensorflow/core/common_runtime/direct_session.cc:296] Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from seqlen256_v1.ckpt/model.ckpt-413000
global_step/IsInitialized/VarIsInitializedOp: (VarIsInitializedOp): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.457504: I tensorflow/core/common_runtime/placer.cc:54] global_step/IsInitialized/VarIsInitializedOp: (V

2020-12-19 10:36:34.479215: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_3/multi_head_attention_3/dense_21/Tensordot/transpose_1: (Transpose)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_3/multi_head_attention_3/dense_21/Tensordot/Reshape_1: (Reshape): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.479229: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_3/multi_head_attention_3/dense_21/Tensordot/Reshape_1: (Reshape)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_3/multi_head_attention_3/dense_21/Tensordot/MatMul: (MatMul): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.479244: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_3/multi_head_attention_3/dense_21/Tensordot/MatMul: (MatMul)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_3/multi_head_attention_3/dense_21/Tensordot: (Reshape): /job:localhost/replica:0/task:0/device:CPU

encoder/encoder_layer_12/multi_head_attention_12/dense_74/Tensordot/MatMul: (MatMul): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.531263: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_12/multi_head_attention_12/dense_74/Tensordot/MatMul: (MatMul)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_12/multi_head_attention_12/dense_74/Tensordot: (Reshape): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.531278: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_12/multi_head_attention_12/dense_74/Tensordot: (Reshape)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_12/multi_head_attention_12/dense_74/BiasAdd/ReadVariableOp: (ReadVariableOp): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.531290: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_12/multi_head_attention_12/dense_74/BiasAdd/ReadVariableOp: (ReadVariableOp)/job:localhost/replica:0

encoder/encoder_layer_21/layer_normalization_42/batchnorm/mul/ReadVariableOp: (ReadVariableOp): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.587284: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_21/layer_normalization_42/batchnorm/mul/ReadVariableOp: (ReadVariableOp)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_21/layer_normalization_42/batchnorm/mul: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.587302: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_21/layer_normalization_42/batchnorm/mul: (Mul)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_21/layer_normalization_42/batchnorm/mul_1: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.587319: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_21/layer_normalization_42/batchnorm/mul_1: (Mul)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_21/layer_norm

2020-12-19 10:36:34.640711: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_29/multi_head_attention_29/dense_177/Tensordot/Reshape_1: (Reshape)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_29/multi_head_attention_29/dense_177/Tensordot/MatMul: (MatMul): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.640726: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_29/multi_head_attention_29/dense_177/Tensordot/MatMul: (MatMul)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_29/multi_head_attention_29/dense_177/Tensordot: (Reshape): /job:localhost/replica:0/task:0/device:CPU:0
2020-12-19 10:36:34.640742: I tensorflow/core/common_runtime/placer.cc:54] encoder/encoder_layer_29/multi_head_attention_29/dense_177/Tensordot: (Reshape)/job:localhost/replica:0/task:0/device:CPU:0
encoder/encoder_layer_29/multi_head_attention_29/dense_177/BiasAdd/ReadVariableOp: (ReadVariableOp): /job:localhost/replica:0/tas

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2020-12-19 10:36:38.781061: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 413000 into seqlen256_v1.ckpt/model.ckpt.
INFO:tensorflow:global_step/sec: 0.0193399
INFO:tensorflow:examples/sec: 0.0773595
INFO:tensorflow:global_step/sec: 0.0316677
INFO:tensorflow:examples/sec: 0.126671
INFO:tensorflow:global_step/sec: 0.0316825
INFO:tensorflow:examples/sec: 0.12673
INFO:tensorflow:global_step/sec: 0.031693
INFO:te

In [97]:
!wget -O generation.py https://raw.githubusercontent.com/salesforce/ctrl/master/generation.py
!wget -O control_codes.py https://raw.githubusercontent.com/salesforce/ctrl/master/control_codes.py
!wget -O vocab https://raw.githubusercontent.com/salesforce/ctrl/master/vocab
!wget -O codes https://raw.githubusercontent.com/salesforce/ctrl/master/codes

--2020-12-19 12:04:13--  https://raw.githubusercontent.com/salesforce/ctrl/master/generation.py
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12530 (12K) [text/plain]
Saving to: ‘generation.py’

generation.py       100%[===================>]  12.24K  --.-KB/s    in 0.001s  

2020-12-19 12:04:13 (17.6 MB/s) - ‘generation.py’ saved [12530/12530]

--2020-12-19 12:04:13--  https://raw.githubusercontent.com/salesforce/ctrl/master/control_codes.py
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239 (1.2K) [text/plain]
Saving to: ‘control_codes.py’

control_codes.py    100%[===================>]   1.21K  --.-KB/s    in 0s      

2020-12-19 12:04:13 (98.5 MB/s) - ‘control_codes.py’ saved [1239/1239]

--2020-12-19 12:04:14--  

In [102]:
!python generation.py --model_dir seqlen256_v1.ckpt/

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tied_embedding_softmax (TiedEmb multiple             315810054   input_1[0][0]                    
                                                                 encoder[0][0]                    
__________________________

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2020-12-19 12:17:44.363974: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
Instructions for updating:
Use standard file utilities to get mtimes.
Loading vocabulary from vocab ...
Read 6086453827 words (246531 unique) from v

---------------------------------------
Legal Trump is a real estate developer and has been for years. He was the one who bought my house in Florida from me when I moved

---------------------------------------
Legal Trump is a real estate developer and has been for years. He was the one who bought my house in Florida from me when I moved there

---------------------------------------
Legal Trump is a real estate developer and has been for years. He was the one who bought my house in Florida from me when I moved there to

---------------------------------------
Legal Trump is a real estate developer and has been for years. He was the one who bought my house in Florida from me when I moved there to be

---------------------------------------
Legal Trump is a real estate developer and has been for years. He was the one who bought my house in Florida from me when I moved there to be with

---------------------------------------
Legal Trump is a real estate developer and has been for years

In [101]:
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import os
import numpy as np
tf.enable_eager_execution()
import transformer
import argparse
import pdb
import sys
import re
from collections import Counter
from tensorflow.python import debug as tf_debug
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import embedding_ops
import fastBPE
import platform
from control_codes import CONTROL_CODES

use_py3 = platform.python_version()[0] == '3'

parser = argparse.ArgumentParser(description='TensorFlow code for generating from CTRL')
parser.add_argument('--model_dir', type=str, default='seqlen256_v1.ckpt/',
                                        help='location of model checkpoint')
parser.add_argument('--seed', type=int, default=1337,
                                        help='random seed for TensorFlow, numpy and PythonHash')
parser.add_argument('--generate_num', type=int, default=256,
                                        help='number of tokens to generate')
parser.add_argument('--temperature', type=float, default=0,
                                        help='temperature for sampling distribution; 0 means greedy')
parser.add_argument('--nucleus', type=float, default=0.,
                                        help='cumulative probability cutoff for nucleus sampling; 0 means no nucleus sampling')
parser.add_argument('--topk', type=int, default=0,
                                        help='topk value for sampling from the softmax distribution ; 0 means no topk preferred')
parser.add_argument('--penalty', type=float, default=1.2,
                                        help='repetition penalty for greedy sampling')
parser.add_argument('--print_once', action='store_true',
                                        help='the completion is printed only at the end; not every word')
parser.add_argument('--topn', type=int, default=0,
                                        help='print top-n candidates during generations; defaults to 0 which is no printing')

args = parser.parse_args()
tf.random.set_random_seed(args.seed)
os.environ['PYTHONHASHSEED'] = str(args.seed)
np.random.seed(args.seed)

# load the vocabulary from file
vocab = open('vocab').read().decode(encoding='utf-8').split('\n') if not use_py3 else open('vocab', encoding='utf-8').read().split('\n')
vocab = list(map(lambda x: x.split(' ')[0], vocab)) + ['<unk>'] + ['\n']
print ('{} unique words'.format(len(vocab)))

# length of the vocabulary
vocab_size = len(vocab)

# define the numericalization map
# idx2word maps the numericalized ID to the word
# word2idx maps the word to the numericalized ID
word2idx = {u:i for i, u in enumerate(vocab)}
idx2word = np.array(vocab)



# sequence length to use for the transformer
# the model is trained with a seq_length of 512
# so, any value <= 512 should work
seq_length = min(args.generate_num, 256)




# the dimension of the transformer
embedding_dim = 1280


# Now, we begin defining the model
# we defer the transformer definition to transformer.py
# here, we only define the tied softmax layer
# this layer ties the softmax weights to the input embeddings
class TiedEmbeddingSoftmax(tf.keras.layers.Layer):

  def __init__(self, vocab_size=vocab_size, embedding_size=embedding_dim, **kwargs):
    super(TiedEmbeddingSoftmax, self).__init__()
    self.w = self.add_weight(name='w', shape=(vocab_size, embedding_size),
                             initializer='random_normal',
                             trainable=True)
    self.b = self.add_weight(name='b', shape=(vocab_size,),
                             initializer='zeros',
                             trainable=True)

  def call(self, inputs, embed=True):
    if embed:
      dtype = tf.keras.backend.dtype(inputs)
      if dtype != 'int32' and dtype != 'int64':
        inputs = math_ops.cast(inputs, 'int32')
      return embedding_ops.embedding_lookup(self.w, inputs)
    else:
      return tf.tensordot(inputs, tf.transpose(self.w), 1) + self.b

# input for the keras model
tokens = tf.keras.layers.Input(shape=(seq_length,), dtype='int32')

# instantiates a tied softmax class
tied_embedding_softmax = TiedEmbeddingSoftmax()

# embedded tokens, before passing it to the transformer
embedded = tied_embedding_softmax(tokens, embed=True)

# the activations after passing it from the transformer
# for some odd reason, TPUs don't play well with specifying the arguments of the Encoder() function
# so you have to leave them at their defaults
transformed = transformer.Encoder()(embedded, training=False)


# pass the activations from our tiedsoftmax class
# this time with embed=False denoting that we are doing the softmax operation
# and not a lookup
logits = tied_embedding_softmax(transformed, embed=False)


# finally, define the Keras model with inputs as tokens and outputs as the logits we just computed
model = tf.keras.Model(inputs=tokens, outputs=logits)


# the loss function is a simple categorical crossentropy between the logits and the labels
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# the optimizer is not used since this code only supports inference
# however, to compile the model, we still define it
optimizer = tf.contrib.tpu.CrossShardOptimizer(
    tf.contrib.estimator.clip_gradients_by_norm(
        tf.train.AdagradOptimizer(learning_rate=1e-2), 0.25)
    )        

# compile the model with the optimizer and loss            
model.compile(optimizer=optimizer, loss=loss)
print(model.summary())


# IMPORTANT
# this is where the saved model is presented to the code
# the model directory should have the model checkpoint and
# a checkpoint file
run_config = tf.contrib.tpu.RunConfig(
        model_dir=args.model_dir)


# this converts the Keras model to a TensorFlow estimator
# this step is critical
# remember to patch the TF 1.14 file before running the code, else you're going to see errors here
estimator_model = tf.keras.estimator.model_to_estimator(keras_model=model, config=run_config)

# we now create a serving function from this estimator
# this enables us to load the model once and easily query it multiple times
def serving_input_fn():
    inputs = {'input_1': tf.placeholder(tf.int32, [1,seq_length])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)
predict_fn = tf.contrib.predictor.from_estimator(estimator_model, serving_input_fn)

# almost there, we now take the user prompt and tokenize with BPE
# load BPE codes
bpe = fastBPE.fastBPE('codes', 'vocab')

temperature = args.temperature
nucleusprob = args.nucleus
penalty = args.penalty
topk = args.topk

while True:
    prompt = raw_input('ENTER PROMPT: ') if not use_py3 else input('ENTER PROMPT: ')
    prompt = prompt.split('\\n') # split on newlines if provided

    # tokenize provided prompt
    split_prompt = ' \n '.join(bpe.apply(prompt))
    split_prompt = split_prompt.split(' ')

    if not any(split_prompt[0] == x for x in CONTROL_CODES.keys()):
        print("WARNING! You are not starting your generation from a control code so you won't get good results")
    text = [word2idx[i] for i in split_prompt]

    # pad with 0s and create a mini-batch of 2 (arbitrary, for ease of code)
    padded_text = text + [0] * (args.generate_num - len(text))
    tokens_generated = np.tile(padded_text, (1,1))
    try:
        for token in range(len(text)-1, args.generate_num-1):
          # get the logits from the prediction function
          # the logic here is a bit convoluted because we are allowing generation past 512 tokens
          # this is done by sliding the window over (past 512 tokens) and continuing prediction
          # I'm sure this can be simplified (TODO)
          if token <= seq_length:
            prompt_logits = predict_fn({'input_1':tokens_generated[:, :seq_length]})['tied_embedding_softmax'].squeeze() / (temperature if temperature>0 else 1.)
            _token = token if token < seq_length else -1
          else:
            _token = -1
            end = token + 1
            start = token - seq_length + 2
            prompt_logits = predict_fn({'input_1':np.hstack((tokens_generated[:,0:1], tokens_generated[:,start:end]))})['tied_embedding_softmax'].squeeze() / (temperature if temperature>0 else 1.)


          # if penalty (for repetition) is non-zero,
          # discount the logits from already generated tokens
          if penalty>0:
              penalized_so_far = set()
              for _ in range(token+1):
                 generated_token = tokens_generated[0][_]
                 # don't penalize newlines
                 # you could also choose not to penalize frequent words
                 # (which incidentally are sorted in the vocab file)
                 # but I don't do that
                 # if it prints too many new lines instead of continuing generating text,
                 # you might want to comment this out
                 if idx2word[generated_token] == '\n':
                     continue
                 if generated_token in penalized_so_far:
                     continue
                 penalized_so_far.add(generated_token)
                 prompt_logits[_token][generated_token] /= penalty

          # disallow some tokens
          prompt_logits[_token][word2idx['<unk>']] = -1e8

          # sometimes, when generating from reddit,
          # it tries to generate the Score (reddit Karma) immediately after generating the Title:
          # to disallow this, we can just prevent it from generating Score
          prompt_logits[_token][word2idx['Sco@@']] = -1e8


          # compute probabilities from logits
          prompt_probs = np.exp(prompt_logits[_token])
          prompt_probs = prompt_probs / sum(prompt_probs)
          pruned_list = np.argsort(prompt_probs)[::-1]
          # if you are using nucleus prob, then compute the nucleus probability size
          if nucleusprob > 0.:
            minimum_topk = 1
            nucleus = max(np.where(np.cumsum(np.sort(prompt_probs)[::-1])>nucleusprob)[0][0], minimum_topk)
          elif topk > 0:
            # we are over-loading notation here
            # if you choose to specify a topk instead of a nucleus,
            # we will hardcode the nucleus to be just that
            nucleus = topk
          else:
            # if you specify neither nucleus or topk,
            # then we will use the whole list
            nucleus = len(pruned_list)
            
          pruned_list = pruned_list[:nucleus]  
          # if you want to disallow more complex tokens, you can do so here
          # for instance, if you want to disallow anything with the phrase `http`,
          # you can delete theme from the pruned_list
          # you can comment this out, I'm keeping it in for demonstration purpose
          tokens_to_disallow = []
          for _ in range(len(pruned_list)):
              if 'http' in idx2word[pruned_list[_]]:
                  tokens_to_disallow.append(_)
          pruned_list = np.delete(pruned_list, tokens_to_disallow)

          if args.topn > 0 :
            print('TOPN :: top-n alternatives:', [idx2word[_] for _ in pruned_list[:args.topn]])

          # if temperature is 0
          # just pick the first (most probable) token
          if temperature==0:
              idx = pruned_list[0]
          else:
            # else,
            # sample from the pruned_list with the logits
            chosen_idx = int(tf.random.categorical(np.expand_dims(prompt_logits[_token][pruned_list],0), num_samples=1).numpy())
            idx = pruned_list[chosen_idx]

          if args.topn > 0 :
            print('TOPN :: chosen word:', idx2word[idx])

          # assign the token for generation
          tokens_generated[0][token+1] = idx

          # clear screen if you want to
          # os.system("clear")


          tokens_generated_so_far = ' '.join([idx2word[c] for c in tokens_generated[0].squeeze()[:token+2]])
          tokens_generated_so_far = re.sub('(@@ )', '', string=tokens_generated_so_far)
          tokens_generated_so_far = re.sub('(@@ ?$)', '', string=tokens_generated_so_far)              

          if not args.print_once:
            print('---------------------------------------')
            print(tokens_generated_so_far)
            print()
        print('---------------------------------------')            
        print(tokens_generated_so_far)
        print()

    except KeyboardInterrupt: #Exception as e:
        print('Continuing')
            


usage: ipykernel_launcher.py [-h] [--model_dir MODEL_DIR] [--seed SEED]
                             [--generate_num GENERATE_NUM]
                             [--temperature TEMPERATURE] [--nucleus NUCLEUS]
                             [--topk TOPK] [--penalty PENALTY] [--print_once]
                             [--topn TOPN]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/lucab/.local/share/jupyter/runtime/kernel-88165fe8-bb08-45ee-afd1-9be917128e4b.json


SystemExit: 2

In [27]:
!cat /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py

cat: /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py: No such file or directory


In [75]:
!python -u convert_tf_to_huggingface_pytorch.py --tf ./seqlen256_v1.ckpt/model.ckpt-413010.data-00000-of-00001 --pytorch pytorch_checkpoint

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/tensorflow/

In [78]:
#!mkdir custom_ctrl_model
#!cd custom_ctrl_model
#!mv pytorch_checkpoint ./custom_ctrl_model
!wget -O config.json https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
!wget -O merges.txt https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
!wget -O vocab.json https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-vocab.json

# run
!python run_generation.py  --model_type ctrl --model_name . --temperature 0.5 --repetition 1.2 --prompt Legal

--2020-12-19 11:29:58--  https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
Resolving storage.googleapis.com... 172.217.9.16, 142.250.68.176, 216.58.193.144, ...
Connecting to storage.googleapis.com|172.217.9.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     611  --.-KB/s    in 0s      

2020-12-19 11:29:58 (17.1 MB/s) - ‘config.json’ saved [611/611]

--2020-12-19 11:29:58--  https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257285 (2.2M) [text/plain]
Saving to: ‘merges.txt’

merges.txt          100%[===================>]   2.15M  --.-KB/s    in 0.06s   

2020-12-19 11:29:59 (36.5 MB/s) - ‘merges.txt’ saved [2257285/2257285]

--2020-12-19 11:29:

In [87]:
#!mkdir custom_ctrl_model
#!cd custom_ctrl_model
#!mv pytorch_checkpoint ./custom_ctrl_model
!wget -O config.json https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
!wget -O merges.txt https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
!wget -O vocab.json https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-vocab.json

# run
!python run_generation.py  --model_type ctrl --model_name ctrl --temperature 0.5 --repetition 1.2 --prompt Legal --prefix Trump

--2020-12-19 11:49:57--  https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
Resolving storage.googleapis.com... 142.250.68.176, 172.217.9.16, 172.217.12.48, ...
Connecting to storage.googleapis.com|142.250.68.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     611  --.-KB/s    in 0s      

2020-12-19 11:49:57 (16.5 MB/s) - ‘config.json’ saved [611/611]

--2020-12-19 11:49:58--  https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257285 (2.2M) [text/plain]
Saving to: ‘merges.txt’

merges.txt          100%[===================>]   2.15M  --.-KB/s    in 0.04s   

2020-12-19 11:49:58 (49.2 MB/s) - ‘merges.txt’ saved [2257285/2257285]

--2020-12-19 11:49

In [81]:
#!mkdir custom_ctrl_model
#!cd custom_ctrl_model
#!mv pytorch_checkpoint ./custom_ctrl_model
!wget -O config.json https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
!wget -O merges.txt https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
!wget -O vocab.json https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-vocab.json

# run
!python run_generation.py  --model_type ctrl --model_name seqlen256_v1.ckpt/model.ckpt-413010.index --temperature 0.5 --repetition 1.2 --prompt Legal

--2020-12-19 11:32:04--  https://storage.googleapis.com/sf-ctrl/pytorch/ctrl-config.json
Resolving storage.googleapis.com... 172.217.9.176, 172.217.1.240, 172.217.14.176, ...
Connecting to storage.googleapis.com|172.217.9.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611 [application/json]
Saving to: ‘config.json’

config.json         100%[===================>]     611  --.-KB/s    in 0s      

2020-12-19 11:32:04 (14.4 MB/s) - ‘config.json’ saved [611/611]

--2020-12-19 11:32:04--  https://raw.githubusercontent.com/salesforce/ctrl/master/ctrl-merges.txt
Resolving raw.githubusercontent.com... 151.101.48.133
Connecting to raw.githubusercontent.com|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2257285 (2.2M) [text/plain]
Saving to: ‘merges.txt’

merges.txt          100%[===================>]   2.15M  --.-KB/s    in 0.1s    

2020-12-19 11:32:05 (21.5 MB/s) - ‘merges.txt’ saved [2257285/2257285]

--2020-12-19 11:32

In [80]:
!ls seqlen256_v1.ckpt

checkpoint				  model.ckpt-413000.index
events.out.tfevents.1608374186.AzureMLAI  model.ckpt-413000.meta
graph.pbtxt				  model.ckpt-413010.data-00000-of-00001
keras					  model.ckpt-413010.index
model.ckpt-413000.data-00000-of-00001	  model.ckpt-413010.meta
